# Infant Respiration Estimation - Google Colab Pipeline

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/J0R0W/Infant-Respiration-Estimation/blob/main/AIR_Complete_Pipeline_Colab.ipynb)

Dieses Notebook enthält alle erforderlichen Schritte zum Ausführen des Infant Respiration Estimation Projekts **in Google Colab**.

## Überblick

Dieses Projekt implementiert automatische Atmungsschätzung bei Säuglingen aus Videos unter Verwendung von Deep Learning-Methoden, insbesondere dem **AIRFlowNet**-Modell.

**Paper:** [Automatic Infant Respiration Estimation from Video: A Deep Flow-based Algorithm and a Novel Public Benchmark](https://arxiv.org/pdf/2307.13110.pdf)

### Unterstützte Modelle:
- **VIRENet** (AIRFlowNet)
- DeepPhys
- EfficientPhys
- TS-CAN

### Datasets:
- **AIR-125**: 125 annotierte Säuglingsvideos
- **COHFACE**: Erwachsenen-Atmungsdatensatz

---

## Wichtige Hinweise für Google Colab:

1. **GPU Runtime**: Stelle sicher, dass du eine GPU-Runtime verwendest
   - `Runtime` → `Change runtime type` → `Hardware accelerator: GPU` (T4 empfohlen)

2. **Session-Dauer**: Colab-Sessions haben eine Zeitbegrenzung. Speichere wichtige Ergebnisse regelmäßig.

3. **Google Drive**: Optional kannst du Google Drive mounten, um Daten zu persistieren.

4. **Erster Durchlauf**: Der erste Durchlauf dauert länger wegen Repository-Klon und Dataset-Download.

## 1. GPU-Überprüfung

Überprüfe, ob eine GPU verfügbar ist. Falls nicht, ändere die Runtime-Einstellungen.

In [ ]:
import sys
import os

print(f"Python Version: {sys.version}")
print(f"Current Working Directory: {os.getcwd()}")

# Prüfe GPU-Verfügbarkeit
!nvidia-smi

# Prüfe CUDA mit PyTorch (falls bereits installiert)
try:
    import torch
    print(f"\nPyTorch Version: {torch.__version__}")
    print(f"CUDA Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA Version: {torch.version.cuda}")
        print(f"GPU Name: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
except ImportError:
    print("\nPyTorch wird in einem späteren Schritt installiert.")

print("\n" + "="*60)
if torch.cuda.is_available():
    print("✓ GPU ist verfügbar! Training wird schneller sein.")
else:
    print("⚠ WARNUNG: Keine GPU verfügbar!")
    print("  Bitte ändere die Runtime: Runtime → Change runtime type → GPU")
print("="*60)

## 2. Google Drive mounten (Optional)

Du kannst Google Drive mounten, um Datasets, Modelle und Ergebnisse zu persistieren.

**Vorteile:**
- Daten bleiben nach Session-Ende erhalten
- Schnellerer Zugriff bei wiederholten Ausführungen
- Teilen von Daten zwischen verschiedenen Notebooks

**Optional:** Überspringe diesen Schritt, wenn du Drive nicht verwenden möchtest.

In [ ]:
from google.colab import drive
import os

# Uncomment die nächste Zeile, um Google Drive zu mounten
# drive.mount('/content/drive')

# Optional: Erstelle ein Projektverzeichnis in Drive
# project_dir = '/content/drive/MyDrive/InfantRespiration'
# os.makedirs(project_dir, exist_ok=True)
# print(f"Projektverzeichnis: {project_dir}")

print("Google Drive Mount übersprungen.")
print("Alle Daten werden im temporären Colab-Speicher abgelegt.")
print("\nHinweis: Daten gehen verloren, wenn die Runtime beendet wird.")

## 3. Repository klonen

Klone das Infant Respiration Estimation Repository von GitHub.

In [ ]:
import os
from pathlib import Path

# Repository URL
repo_url = "https://github.com/J0R0W/Infant-Respiration-Estimation.git"
repo_name = "Infant-Respiration-Estimation"
repo_path = Path(f"/content/{repo_name}")

# Klone das Repository, falls es noch nicht existiert
if not repo_path.exists():
    print(f"Klone Repository von {repo_url}...")
    !git clone {repo_url} {repo_path}
    print(f"\n✓ Repository geklont: {repo_path}")
else:
    print(f"Repository existiert bereits: {repo_path}")
    print("Aktualisiere Repository...")
    !cd {repo_path} && git pull

# Wechsle ins Repository-Verzeichnis
os.chdir(repo_path)
print(f"\nCurrent Working Directory: {os.getcwd()}")

# Zeige Repository-Struktur
print("\nRepository-Struktur:")
!ls -la

# Füge Repository zum Python-Path hinzu
if str(repo_path) not in sys.path:
    sys.path.insert(0, str(repo_path))
    print(f"\n✓ {repo_path} zum Python-Path hinzugefügt")

## 4. Installation der Abhängigkeiten

Installiere alle erforderlichen Python-Pakete.

**Hinweis:** Dieser Schritt kann 5-10 Minuten dauern.

In [ ]:
# Installiere PyTorch zuerst (falls nicht bereits vorhanden)
print("Überprüfe PyTorch-Installation...")
!pip install -q torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116

# Installiere requirements aus der requirements.txt
print("\nInstalliere Projekt-Abhängigkeiten...")
!pip install -q -r requirements.txt

# Zusätzliche Pakete für Notebook-Visualisierung
!pip install -q ipywidgets

print("\n✓ Alle Abhängigkeiten installiert!")

# Verifiziere die Installation
print("\nVerifiziere Installation...")
import torch
import numpy as np
import cv2
import yaml
import pandas as pd
import matplotlib.pyplot as plt

print(f"✓ PyTorch {torch.__version__}")
print(f"✓ NumPy {np.__version__}")
print(f"✓ OpenCV {cv2.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")

## 5. Dataset Download und Extraktion

Lade den AIR-125 Dataset herunter und extrahiere ihn.

**Hinweis:** 
- Dataset-Größe: ~mehrere GB
- Download kann 10-30 Minuten dauern, je nach Verbindung
- Falls Google Drive gemountet ist, kannst du den Pfad anpassen, um den Dataset dort zu speichern

In [ ]:
import urllib.request
import zipfile
from pathlib import Path
from tqdm.notebook import tqdm

# Definiere Pfade
# Option 1: In Colab-Speicher (temporär)
dataset_url = "https://coe.northeastern.edu/Research/AClab/AIR-125/AIR.zip"
download_path = Path("/content/data/AIR.zip")
extract_path = Path("/content/data/AIR")

# Option 2: In Google Drive (persistent) - uncomment falls Drive gemountet
# download_path = Path("/content/drive/MyDrive/InfantRespiration/data/AIR.zip")
# extract_path = Path("/content/drive/MyDrive/InfantRespiration/data/AIR")

# Erstelle Verzeichnis
download_path.parent.mkdir(parents=True, exist_ok=True)

# Download mit Fortschrittsanzeige
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

if not download_path.exists():
    print(f"Downloading AIR-125 dataset from {dataset_url}...")
    print("Dies kann 10-30 Minuten dauern, je nach Internetgeschwindigkeit.")
    print("☕ Zeit für einen Kaffee!\n")
    
    with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc='AIR.zip') as t:
        urllib.request.urlretrieve(dataset_url, download_path, reporthook=t.update_to)
    print(f"\n✓ Download abgeschlossen: {download_path}")
else:
    print(f"✓ Dataset bereits heruntergeladen: {download_path}")
    print(f"  Größe: {download_path.stat().st_size / 1e9:.2f} GB")

# Extrahiere ZIP-Datei
if not extract_path.exists():
    print(f"\nExtrahiere Dataset nach {extract_path}...")
    print("Dies kann einige Minuten dauern...")
    with zipfile.ZipFile(download_path, 'r') as zip_ref:
        # Extrahiere mit Fortschrittsanzeige
        members = zip_ref.namelist()
        for member in tqdm(members, desc='Extracting'):
            zip_ref.extract(member, extract_path.parent)
    print("✓ Extraktion abgeschlossen!")
else:
    print(f"✓ Dataset bereits extrahiert: {extract_path}")

# Zeige Dataset-Struktur
print("\nDataset-Struktur:")
!du -sh {extract_path}
print("\nInhalt:")
!ls -lh {extract_path} | head -15

## 6. Dataset-Inspektion

Schaue dir die Dataset-Struktur und einige Sample-Daten an.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Verwende den gleichen Pfad wie beim Download
dataset_path = Path("/content/data/AIR")
# dataset_path = Path("/content/drive/MyDrive/InfantRespiration/data/AIR")  # Falls Drive verwendet

if dataset_path.exists():
    # Zähle Dateien
    video_files = list(dataset_path.rglob('*.avi')) + list(dataset_path.rglob('*.mp4'))
    mat_files = list(dataset_path.rglob('*.mat'))
    
    print(f"Dataset-Statistiken:")
    print(f"  Anzahl Video-Dateien: {len(video_files)}")
    print(f"  Anzahl MAT-Dateien (Annotationen): {len(mat_files)}")
    
    if video_files:
        print("\nBeispiel Video-Dateien:")
        for vf in video_files[:5]:
            file_size = vf.stat().st_size / 1e6
            print(f"  - {vf.name} ({file_size:.1f} MB)")
        
        # Lade ein Sample-Video
        sample_video = video_files[0]
        print(f"\nLade Sample-Video: {sample_video.name}")
        
        cap = cv2.VideoCapture(str(sample_video))
        ret, frame = cap.read()
        
        if ret:
            # Zeige mehrere Frames
            frames_to_show = 4
            fig, axes = plt.subplots(1, frames_to_show, figsize=(20, 5))
            
            frame_indices = np.linspace(0, int(cap.get(cv2.CAP_PROP_FRAME_COUNT))-1, frames_to_show, dtype=int)
            
            for idx, frame_idx in enumerate(frame_indices):
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                ret, frame = cap.read()
                if ret:
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    axes[idx].imshow(frame_rgb)
                    axes[idx].set_title(f"Frame {frame_idx}")
                    axes[idx].axis('off')
            
            plt.suptitle(f"Sample Frames from {sample_video.name}", fontsize=14)
            plt.tight_layout()
            plt.show()
            
            # Video-Informationen
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            duration = frame_count / fps if fps > 0 else 0
            
            print(f"\nVideo-Informationen:")
            print(f"  FPS: {fps:.2f}")
            print(f"  Frames: {frame_count}")
            print(f"  Auflösung: {width}x{height}")
            print(f"  Dauer: {duration:.2f} Sekunden")
        
        cap.release()
    else:
        print("\n⚠ Keine Video-Dateien gefunden!")
else:
    print(f"⚠ Dataset-Pfad nicht gefunden: {dataset_path}")
    print("Bitte führe Zelle 5 aus, um das Dataset herunterzuladen.")

## 7. Konfiguration erstellen

Erstelle eine Konfigurationsdatei für das Training mit dem AIR-125 Dataset.

In [ ]:
import yaml
from pathlib import Path
import os

# Stelle sicher, dass wir im Repository-Verzeichnis sind
repo_path = Path("/content/Infant-Respiration-Estimation")
os.chdir(repo_path)

# Definiere Pfade (verwende die gleichen wie beim Download)
raw_data_path = "/content/data/AIR"
processed_data_path = "/content/data/processed_AIR"

# Falls Google Drive verwendet wird, uncomment:
# raw_data_path = "/content/drive/MyDrive/InfantRespiration/data/AIR"
# processed_data_path = "/content/drive/MyDrive/InfantRespiration/data/processed_AIR"

# Erstelle processed data Verzeichnis
Path(processed_data_path).mkdir(parents=True, exist_ok=True)

# Erstelle Konfiguration für VIRENet
config = {
    'BASE': [''],
    'TOOLBOX_MODE': 'train_and_test',
    'TRAIN': {
        'BATCH_SIZE': 4,
        'EPOCHS': 10,
        'LR': 0.001,
        'MODEL_FILE_NAME': 'AIR_VIRENet_Colab',
        'DATA': {
            'FS': 5,
            'DATASET': 'AIR',
            'DO_PREPROCESS': True,
            'DATA_FORMAT': 'NDCHW',
            'DATA_PATH': raw_data_path,
            'CACHED_PATH': processed_data_path,
            'EXP_DATA_NAME': '',
            'BEGIN': 0.0,
            'END': 0.7,
            'PREPROCESS': {
                'DATA_TYPE': ['Standardized'],
                'LABEL_TYPE': 'Standardized',
                'DO_CHUNK': True,
                'CHUNK_LENGTH': 60,
                'DYNAMIC_DETECTION': False,
                'DYNAMIC_DETECTION_FREQUENCY': 60,
                'CROP_FACE': False,
                'LARGE_FACE_BOX': True,
                'LARGE_BOX_COEF': 1.5,
                'H': 96,
                'W': 96
            }
        }
    },
    'VALID': {
        'DATA': {
            'FS': 5,
            'DATASET': 'AIR',
            'DO_PREPROCESS': True,
            'DATA_FORMAT': 'NDCHW',
            'DATA_PATH': raw_data_path,
            'CACHED_PATH': processed_data_path,
            'EXP_DATA_NAME': '',
            'BEGIN': 0.7,
            'END': 1.0,
            'PREPROCESS': {
                'DATA_TYPE': ['Standardized'],
                'LABEL_TYPE': 'Standardized',
                'DO_CHUNK': True,
                'CHUNK_LENGTH': 60,
                'DYNAMIC_DETECTION': False,
                'DYNAMIC_DETECTION_FREQUENCY': 60,
                'CROP_FACE': False,
                'LARGE_FACE_BOX': True,
                'LARGE_BOX_COEF': 1.5,
                'H': 96,
                'W': 96
            }
        }
    },
    'TEST': {
        'METRICS': ['MAE', 'RMSE', 'MAPE', 'Pearson'],
        'USE_LAST_EPOCH': False,
        'DATA': {
            'FS': 5,
            'DATASET': 'AIR',
            'DO_PREPROCESS': True,
            'DATA_FORMAT': 'NDCHW',
            'DATA_PATH': raw_data_path,
            'CACHED_PATH': processed_data_path,
            'EXP_DATA_NAME': '',
            'BEGIN': 0.0,
            'END': 1.0,
            'PREPROCESS': {
                'DATA_TYPE': ['Standardized'],
                'LABEL_TYPE': 'Standardized',
                'DO_CHUNK': True,
                'CHUNK_LENGTH': 60,
                'DYNAMIC_DETECTION': False,
                'DYNAMIC_DETECTION_FREQUENCY': 60,
                'CROP_FACE': False,
                'LARGE_FACE_BOX': True,
                'LARGE_BOX_COEF': 1.5,
                'H': 96,
                'W': 96
            }
        }
    },
    'DEVICE': 'cuda:0',
    'NUM_OF_GPU_TRAIN': 1,
    'LOG': {
        'PATH': 'runs/exp'
    },
    'MODEL': {
        'DROP_RATE': 0.2,
        'NAME': 'VIRENet',
        'MODEL_DIR': '',
        'VIRENET': {
            'FRAME_DEPTH': 10
        }
    },
    'INFERENCE': {
        'BATCH_SIZE': 4,
        'EVALUATION_METHOD': 'FFT',
        'MODEL_PATH': ''
    }
}

# Speichere Konfiguration
config_path = repo_path / 'colab_config.yaml'
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"✓ Konfiguration erstellt: {config_path}")
print(f"\nDataset-Pfad: {raw_data_path}")
print(f"Verarbeiteter Dataset-Pfad: {processed_data_path}")
print(f"\nKonfiguration:")
print(f"  Modell: {config['MODEL']['NAME']}")
print(f"  Batch Size: {config['TRAIN']['BATCH_SIZE']}")
print(f"  Epochen: {config['TRAIN']['EPOCHS']}")
print(f"  Learning Rate: {config['TRAIN']['LR']}")
print(f"  Device: {config['DEVICE']}")
print(f"\nWorking Directory: {os.getcwd()}")

## 8. Model Training

Trainiere das VIRENet (AIRFlowNet) Modell auf dem AIR-125 Dataset.

**Wichtige Hinweise:**
- **Erster Run**: Preprocessing wird durchgeführt (kann 1-2 Stunden dauern)
- **Training**: 10 Epochen können 2-4 Stunden dauern (je nach GPU)
- **GPU-Empfehlung**: T4 oder besser
- **Session-Timeout**: Colab-Sessions können nach 12 Stunden oder bei Inaktivität unterbrochen werden
- **Tipp**: Speichere Zwischenergebnisse in Google Drive für Persistenz

### Training-Fortschritt überwachen:
- Loss-Werte sollten sinken
- Validierungs-Metriken werden nach jeder Epoche angezeigt
- Checkpoints werden automatisch gespeichert

In [ ]:
import os
from pathlib import Path

# Stelle sicher, dass wir im Repository-Verzeichnis sind
repo_path = Path("/content/Infant-Respiration-Estimation")
os.chdir(repo_path)

print(f"Working Directory: {os.getcwd()}")
print(f"Config File: {repo_path / 'colab_config.yaml'}")
print("\n" + "="*60)
print("STARTE TRAINING")
print("="*60)
print("\nDies kann mehrere Stunden dauern...")
print("☕ Zeit für Pausen!\n")

# Führe Training aus
!python main.py --config_file ./colab_config.yaml

## 9. Model Testing / Inference

Nach dem Training evaluieren wir das Modell auf den Testdaten.

In [ ]:
import yaml
import os
from pathlib import Path

repo_path = Path("/content/Infant-Respiration-Estimation")
os.chdir(repo_path)

# Lade bestehende Konfiguration
with open('colab_config.yaml', 'r') as f:
    test_config = yaml.safe_load(f)

# Ändere Modus auf "only_test"
test_config['TOOLBOX_MODE'] = 'only_test'

# Finde das trainierte Modell
model_dir = Path('./runs/exp/AIR_VIRENet_Colab')

if model_dir.exists():
    model_files = list(model_dir.glob('*.pth'))
    if model_files:
        # Verwende das neueste Modell
        latest_model = max(model_files, key=os.path.getctime)
        test_config['INFERENCE']['MODEL_PATH'] = str(latest_model)
        print(f"✓ Verwende Modell: {latest_model.name}")
    else:
        print("⚠ Keine .pth Modell-Dateien gefunden.")
        print("Verfügbare Dateien im Modell-Verzeichnis:")
        !ls -lh {model_dir}
else:
    print(f"⚠ Modell-Verzeichnis nicht gefunden: {model_dir}")
    print("Bitte führe zuerst das Training aus (Zelle 8).")

# Speichere Test-Konfiguration
test_config_path = repo_path / 'colab_test_config.yaml'
with open(test_config_path, 'w') as f:
    yaml.dump(test_config, f, default_flow_style=False)

print(f"\n✓ Test-Konfiguration erstellt: {test_config_path}")
print("\n" + "="*60)
print("STARTE TESTING")
print("="*60 + "\n")

# Führe Testing aus
!python main.py --config_file ./colab_test_config.yaml

## 10. Ergebnisse visualisieren

Visualisiere die Trainingsergebnisse und Evaluations-Metriken.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Finde Ergebnisdateien
results_dir = Path('/content/Infant-Respiration-Estimation/runs/exp')

if results_dir.exists():
    print("Suche nach Ergebnisdateien...\n")
    
    # Suche nach CSV-Dateien
    csv_files = list(results_dir.rglob('*.csv'))
    
    if csv_files:
        print(f"✓ Gefunden: {len(csv_files)} Ergebnisdateien\n")
        
        for csv_file in csv_files[:5]:  # Zeige bis zu 5 Dateien
            print("="*70)
            print(f"📊 {csv_file.name}")
            print("="*70)
            
            try:
                df = pd.read_csv(csv_file)
                print(df.to_string())
                print()
                
                # Visualisiere numerische Spalten
                numeric_cols = df.select_dtypes(include=[np.number]).columns
                if len(numeric_cols) > 0:
                    n_cols = min(len(numeric_cols), 4)
                    fig, axes = plt.subplots(1, n_cols, figsize=(5*n_cols, 4))
                    if n_cols == 1:
                        axes = [axes]
                    
                    for idx, col in enumerate(numeric_cols[:n_cols]):
                        axes[idx].plot(df[col], marker='o', linewidth=2, markersize=6)
                        axes[idx].set_title(col, fontsize=12, fontweight='bold')
                        axes[idx].set_xlabel('Sample/Epoch', fontsize=10)
                        axes[idx].set_ylabel('Value', fontsize=10)
                        axes[idx].grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
                    
                # Statistiken
                if len(numeric_cols) > 0:
                    print("\n📈 Statistiken:")
                    print(df[numeric_cols].describe())
                    print()
                
            except Exception as e:
                print(f"⚠ Fehler beim Laden: {e}\n")
    else:
        print("⚠ Keine CSV-Ergebnisdateien gefunden.")
        print("\nVerfügbare Dateien:")
        !find {results_dir} -type f -name "*" | head -20
else:
    print(f"⚠ Ergebnisverzeichnis nicht gefunden: {results_dir}")
    print("Stelle sicher, dass das Training abgeschlossen wurde.")

## 11. TensorBoard (Optional)

Visualisiere Training-Logs mit TensorBoard (falls vorhanden).

In [ ]:
# Lade TensorBoard Extension
%load_ext tensorboard

# Starte TensorBoard
log_dir = '/content/Infant-Respiration-Estimation/runs/exp'

print(f"Starte TensorBoard...")
print(f"Log-Verzeichnis: {log_dir}\n")

%tensorboard --logdir {log_dir}

## 12. Ergebnisse sichern (Optional)

Speichere wichtige Ergebnisse und Modelle in Google Drive oder lade sie herunter.

In [ ]:
import shutil
from pathlib import Path
from google.colab import files

results_dir = Path('/content/Infant-Respiration-Estimation/runs/exp')

if results_dir.exists():
    print("Option 1: Ergebnisse herunterladen")
    print("="*60)
    
    # Erstelle ZIP-Archiv
    archive_path = Path('/content/training_results')
    shutil.make_archive(str(archive_path), 'zip', results_dir)
    
    zip_file = str(archive_path) + '.zip'
    zip_size = Path(zip_file).stat().st_size / 1e6
    print(f"✓ Archiv erstellt: {zip_file}")
    print(f"  Größe: {zip_size:.2f} MB")
    
    # Uncomment um herunterzuladen
    # files.download(zip_file)
    # print("\n✓ Download gestartet!")
    
    print("\n" + "="*60)
    print("Option 2: Nach Google Drive kopieren")
    print("="*60)
    print("Falls Drive gemountet ist, uncomment die folgenden Zeilen:")
    print()
    print("# drive_path = Path('/content/drive/MyDrive/InfantRespiration/results')")
    print("# drive_path.mkdir(parents=True, exist_ok=True)")
    print("# shutil.copytree(results_dir, drive_path / 'exp', dirs_exist_ok=True)")
    print("# print(f'✓ Ergebnisse nach Drive kopiert: {drive_path}')")
    
else:
    print(f"⚠ Keine Ergebnisse gefunden in: {results_dir}")

## 13. Andere Modelle ausprobieren

Experimentiere mit verschiedenen Modellen.

In [ ]:
# Verfügbare Modelle
available_models = {
    'VIRENet': {
        'description': 'AIRFlowNet - Flow-basiertes Modell (empfohlen für Säuglinge)',
        'paper': 'https://arxiv.org/pdf/2307.13110.pdf',
        'best_for': 'Infant respiration estimation'
    },
    'DeepPhys': {
        'description': 'DeepPhys - Attention-basiertes Modell',
        'paper': 'https://arxiv.org/abs/1805.07888',
        'best_for': 'General physiological signal estimation'
    },
    'EfficientPhys': {
        'description': 'EfficientPhys - Effizientes Modell',
        'paper': 'WACV 2023',
        'best_for': 'Fast and accurate cardiac measurement'
    },
    'Tscan': {
        'description': 'TS-CAN - Temporal Shift Attention Network',
        'paper': 'NeurIPS 2020',
        'best_for': 'On-device contactless vitals measurement'
    }
}

print("🤖 VERFÜGBARE MODELLE")
print("="*70)

for model_name, info in available_models.items():
    print(f"\n📦 {model_name}")
    print(f"   {info['description']}")
    print(f"   Optimal für: {info['best_for']}")
    print(f"   Paper: {info['paper']}")

print("\n" + "="*70)
print("\n💡 Um ein anderes Modell zu verwenden:")
print("   1. Gehe zurück zu Zelle 7 (Konfiguration erstellen)")
print("   2. Ändere config['MODEL']['NAME'] zum gewünschten Modell")
print("   3. Führe Zellen 7-8 erneut aus für Training")
print("\n⚡ Quick-Test mit anderem Modell:")

# Funktion zum schnellen Modellwechsel
def create_config_for_model(model_name):
    """
    Erstelle Config für spezifisches Modell.
    Uncomment und führe aus um zu testen.
    """
    print(f"\nErstelle Konfiguration für {model_name}...")
    # Code hier...

print("\n# Beispiel:")
print("# create_config_for_model('DeepPhys')")

## 14. Zusammenfassung und nächste Schritte

### ✅ Was wir getan haben:

1. ✓ GPU überprüft und Colab-Umgebung eingerichtet
2. ✓ Repository geklont
3. ✓ Abhängigkeiten installiert
4. ✓ AIR-125 Dataset heruntergeladen und extrahiert
5. ✓ Konfigurationsdatei erstellt
6. ✓ Modell trainiert (VIRENet)
7. ✓ Modell getestet und evaluiert
8. ✓ Ergebnisse visualisiert

### 🚀 Nächste Schritte:

#### Experimente:
- **Hyperparameter-Tuning**: Experimentiere mit Learning Rate, Batch Size, Epochs
- **Andere Modelle**: Probiere DeepPhys, EfficientPhys oder TS-CAN
- **Data Augmentation**: Erweitere das Preprocessing
- **COHFACE Dataset**: Evaluiere auf Erwachsenen-Daten

#### Optimierung:
- **Längeres Training**: Erhöhe Anzahl der Epochen
- **Mixed Precision**: Nutze fp16 für schnelleres Training
- **Größere Batch Size**: Falls GPU-Memory ausreicht

#### Deployment:
- **Model Export**: Exportiere zu ONNX oder TorchScript
- **Inference-Optimierung**: Reduziere Modellgröße
- **Eigene Videos**: Teste auf eigenen Säuglingsvideos

### 📝 Wichtige Konfigurationsparameter:

```yaml
TRAIN:
  BATCH_SIZE: 4          # Erhöhen für schnelleres Training (wenn GPU-Memory erlaubt)
  EPOCHS: 10             # Mehr Epochen = bessere Konvergenz
  LR: 0.001              # Learning Rate anpassen für stabileres Training

PREPROCESS:
  CHUNK_LENGTH: 60       # Video-Chunk-Länge in Frames
  H: 96                  # Frame-Höhe (höher = mehr Details, mehr Rechenzeit)
  W: 96                  # Frame-Breite
  CROP_FACE: False       # Aktivieren für Gesichtsfokus

MODEL:
  NAME: VIRENet          # Modellauswahl
  DROP_RATE: 0.2         # Dropout für Regularisierung
```

### 📚 Ressourcen:

- **Paper**: [Automatic Infant Respiration Estimation from Video](https://arxiv.org/pdf/2307.13110.pdf)
- **GitHub**: [Infant-Respiration-Estimation](https://github.com/J0R0W/Infant-Respiration-Estimation)
- **Original rPPG-Toolbox**: [ubicomplab/rPPG-Toolbox](https://github.com/ubicomplab/rPPG-Toolbox)
- **Dataset**: [AIR-125](https://coe.northeastern.edu/Research/AClab/AIR-125/)

### 💾 Datensicherung:

**WICHTIG**: Colab-Sessions sind temporär!

- Speichere wichtige Ergebnisse in Google Drive
- Lade trainierte Modelle herunter
- Exportiere Evaluations-Metriken

### 🐛 Troubleshooting:

**Problem: CUDA out of memory**
- Lösung: Reduziere `BATCH_SIZE` in der Config

**Problem: Session Timeout**
- Lösung: Aktiviere Google Drive für Persistenz, nutze Colab Pro

**Problem: Dataset Download fehlgeschlagen**
- Lösung: Prüfe Internetverbindung, versuche erneut

**Problem: Import Errors**
- Lösung: Führe Zelle 4 (Installation) erneut aus

### 📧 Support:

Bei Fragen oder Problemen:
- Siehe README.md im Repository
- Öffne ein Issue auf GitHub
- Konsultiere das Original-Paper

---

**Viel Erfolg mit deinen Experimenten! 🚀**